# Zero-Shot Classification: mDeBERTa
## Model: MoritzLaurer/mDeBERTa-v3-base-mnli-xnli

Zero-Shot NLI-basierte Klassifikation auf Volltext.

**Voraussetzung:** `init_data.ipynb` muss vorher ausgeführt worden sein.

In [ ]:
# ===== MODEL CONFIG =====
MODEL_ID = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
MODEL_SHORT_NAME = "mdeberta"
MODEL_TYPE = "zero-shot"  # "zero-shot" | "few-shot" | "fine-tuned"

# Welcher Split wird evaluiert?
EVALUATE_ON = "test"  # "test" oder "eval"

# Batch-Größe für Klassifikation (kleiner = weniger VRAM, langsamer)
BATCH_SIZE = 8

# ===== NLI CONFIG =====
# Hypothesis Template für Zero-Shot NLI
HYPOTHESIS_TEMPLATE = "Dieser Text handelt von {}."

# Candidate Labels: None = Labels aus init_data.ipynb verwenden
# Oder eigene Liste angeben: ["Label A", "Label B", ...]
CANDIDATE_LABELS = None

# ===== MODEL INFO (für Report) =====
MODEL_INFO = {
    "huggingface_id": MODEL_ID,
    "language": "Multilingual (100+ Sprachen, inkl. Deutsch)",
    "max_tokens": 512,
    "parameters": "278M",
    "notes": "mDeBERTa unterstützt KEIN FP16 (NaN). Immer FP32 verwenden.",
}

In [ ]:
# Imports + Daten laden
import importlib
import warnings

import torch
from tqdm.auto import tqdm
from transformers import pipeline as hf_pipeline

import pipeline_utils as pu
importlib.reload(pu)

warnings.filterwarnings("ignore")

# Daten aus Runtime laden
data = pu.get_runtime_data()
eval_df = data[EVALUATE_ON]

# Labels bestimmen
if CANDIDATE_LABELS is None:
    CANDIDATE_LABELS = list(data["label_mapping"].values())

print(f"Evaluiere auf '{EVALUATE_ON}' Split: {len(eval_df)} Artikel")
print(f"Labels: {len(CANDIDATE_LABELS)}")
print(f"GPU: {torch.cuda.is_available()}")

In [ ]:
# Modell laden
device = 0 if torch.cuda.is_available() else -1

classifier = hf_pipeline(
    "zero-shot-classification",
    model=MODEL_ID,
    device=device,
)

print(f"Modell geladen: {MODEL_ID}")
print(f"Device: {'GPU' if device == 0 else 'CPU'}")
print(f"Tokenizer max length: {classifier.tokenizer.model_max_length}")

In [ ]:
# Klassifikation (Volltext)
def classify_batch(texts, batch_size=BATCH_SIZE):
    """Zero-Shot Klassifikation mit Progress-Tracking."""
    predictions = [None] * len(texts)
    non_empty_indices = [i for i, t in enumerate(texts) if t.strip()]
    non_empty_texts = [texts[i] for i in non_empty_indices]

    for start in tqdm(range(0, len(non_empty_texts), batch_size), desc="Classifying"):
        batch_texts = non_empty_texts[start:start + batch_size]
        batch_indices = non_empty_indices[start:start + batch_size]

        results = classifier(
            batch_texts,
            candidate_labels=CANDIDATE_LABELS,
            hypothesis_template=HYPOTHESIS_TEMPLATE,
            multi_label=False,
        )
        if isinstance(results, dict):
            results = [results]

        for idx, r in zip(batch_indices, results):
            predictions[idx] = r["labels"][0]

    empty_count = sum(1 for p in predictions if p is None)
    if empty_count > 0:
        print(f"  {empty_count} leere Texte -> 'Andere'")
    return [p if p is not None else "Andere" for p in predictions]


texts = eval_df["text"].fillna("").tolist()
true_labels = eval_df["label"].tolist()

timer = pu.ExperimentTimer()
with timer:
    predictions = classify_batch(texts)

print(f"\nKlassifikation abgeschlossen: {timer.duration_formatted}")
print(f"Durchsatz: {timer.articles_per_second(len(texts)):.2f} Artikel/Sekunde")

In [ ]:
# Evaluation
metrics = pu.evaluate(
    true_labels,
    predictions,
    labels=CANDIDATE_LABELS,
    experiment_name=EVALUATE_ON,
)

pu.print_metrics(metrics, f"Zero-Shot mDeBERTa — {EVALUATE_ON} Split")

In [ ]:
# Confusion Matrix
pu.plot_confusion_matrix(
    metrics,
    title=f"Zero-Shot mDeBERTa ({EVALUATE_ON})",
)

In [ ]:
# Report generieren
report_path = pu.generate_report(
    model_name=f"{MODEL_SHORT_NAME}_zeroshot",
    model_type=MODEL_TYPE,
    metrics=metrics,
    timer=timer,
    model_info=MODEL_INFO,
    candidate_labels=CANDIDATE_LABELS,
    hypothesis_template=HYPOTHESIS_TEMPLATE,
    experiment_notes=(
        "Zero-Shot NLI-Klassifikation auf Volltext. "
        "Texte werden automatisch auf 512 Tokens gekürzt (inverted pyramid)."
    ),
)

print(f"\nReport gespeichert: {report_path}")

In [ ]:
# Summary
print("=" * 70)
print(f"  Model:           {MODEL_ID}")
print(f"  Type:            {MODEL_TYPE}")
print(f"  Split:           {EVALUATE_ON} ({len(eval_df)} Artikel)")
print(f"  F1 Macro:        {metrics['f1_macro']:.4f}")
print(f"  F1 Weighted:     {metrics['f1_weighted']:.4f}")
print(f"  Accuracy:        {metrics['accuracy']:.4f}")
print(f"  Dauer:           {timer.duration_formatted}")
print(f"  Artikel/Sek:     {timer.articles_per_second(len(eval_df)):.2f}")
print("=" * 70)